In [38]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import math
import numpy as np

In [2]:
import os
x = os.walk(".")
# [print(y) for y in x]

In [3]:
!ls

main.ipynb	       tabular-playground-series-mar-2021.zip  toolbox
sample_submission.csv  test.csv				       train.csv


In [4]:
# Add data to memory for discovery
train,test = pd.read_csv('./train.csv'),pd.read_csv('./test.csv')

In [5]:
# Get data structure
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 32 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      300000 non-null  int64  
 1   cat0    300000 non-null  object 
 2   cat1    300000 non-null  object 
 3   cat2    300000 non-null  object 
 4   cat3    300000 non-null  object 
 5   cat4    300000 non-null  object 
 6   cat5    300000 non-null  object 
 7   cat6    300000 non-null  object 
 8   cat7    300000 non-null  object 
 9   cat8    300000 non-null  object 
 10  cat9    300000 non-null  object 
 11  cat10   300000 non-null  object 
 12  cat11   300000 non-null  object 
 13  cat12   300000 non-null  object 
 14  cat13   300000 non-null  object 
 15  cat14   300000 non-null  object 
 16  cat15   300000 non-null  object 
 17  cat16   300000 non-null  object 
 18  cat17   300000 non-null  object 
 19  cat18   300000 non-null  object 
 20  cont0   300000 non-null  float64
 21  cont1   30

In [6]:
# Peek at data examples
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,1,A,I,A,A,E,BI,K,W,AD,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,2,A,K,A,A,E,BI,A,E,BM,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,3,A,K,A,C,E,BI,A,Y,AD,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,4,A,I,G,B,E,BI,C,G,Q,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [57]:
# Get number of unique categories
#Build sample item
cal_cols = train.select_dtypes('object')

cat_stats = np.array([train[c].nunique() for c in cal_cols])
median_cat = math.ceil(np.median(cat_stats))
target_cat_index = int(np.where(cat_stats == median_cat)[0])
target_cat = cal_cols[target_cat_index]
print(target_cat)

for c in train.select_dtypes('object'): print(c,train[c].nunique())

NameError: name 'OrderedDict' is not defined

In [8]:
# Index categories
categoricals_to_ix = {}

for c in train.select_dtypes('object'):
    categoricals_to_ix[c] = {cat:i for i,cat in enumerate(train[c].unique())}

print("cat7 indices:\n",categoricals_to_ix['cat7'])
print("\n")
print("number of cat7 indices:\n",len(categoricals_to_ix['cat7']))

cat7 indices:
 {'S': 0, 'W': 1, 'E': 2, 'Y': 3, 'G': 4, 'AV': 5, 'AF': 6, 'AK': 7, 'AH': 8, 'AN': 9, 'J': 10, 'H': 11, 'U': 12, 'AG': 13, 'AS': 14, 'AA': 15, 'B': 16, 'F': 17, 'AI': 18, 'K': 19, 'AD': 20, 'D': 21, 'N': 22, 'I': 23, 'AW': 24, 'AY': 25, 'AX': 26, 'L': 27, 'AL': 28, 'C': 29, 'R': 30, 'P': 31, 'AU': 32, 'O': 33, 'AR': 34, 'AO': 35, 'AE': 36, 'AP': 37, 'A': 38, 'V': 39, 'Q': 40, 'AQ': 41, 'X': 42, 'BA': 43, 'AM': 44, 'AC': 45, 'M': 46, 'AT': 47, 'T': 48, 'AB': 49, 'AJ': 50}


number of cat7 indices:
 51


In [9]:
# Test embeddings
embeds = {}

for cix_key in categoricals_to_ix.keys():
    cix = categoricals_to_ix[cix_key]
    bag_size = len(cix)
    embed_size = math.isqrt(bag_size)
    embeds[cix_key] = nn.Embedding(bag_size,embed_size)
    
testor = torch.tensor([categoricals_to_ix['cat7']['AG']], dtype=torch.long)
print(embeds['cat7'](testor))

tensor([[ 0.8868,  1.3701, -0.1184, -1.1229,  0.0215, -0.2195,  0.3043]],
       grad_fn=<EmbeddingBackward>)
